How the notebook works: 
Before each cell is explained which figure or table is generated as well as which function from the .py file is used.
Use ctrl + enter to run each cell

1. The necessary library are loaded in as well as the functions.py file as 'FT'

In [ ]:
### Importing libraries

import xarray as xr
import intake
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import datetime
from netCDF4 import Dataset
import Functions as FT

2. The initial datasets are loaded, filtered and combined into one dataset.


In [ ]:
### loading data

cat = intake.open_catalog("https://tcodata.mpimet.mpg.de/catalog.yaml")
Data_AC_B = xr.open_dataset('MetopB-ASCAT-at-BCO-asc-2016-2023.nc') ### extended Ascat data

### lidar datasets
coral_b_data = cat.BCO.lidar_CORAL_LR_b_c1_v1.to_dask() ### coral lidar data
licht_b_data = cat.BCO.lidar_LICHT_LR_b_c2_v1.to_dask() ### licht lidar data

### measurement time dataset and mean divergence of grid
Data_AC_B_time =  Data_AC_B['measurement_time'].isel(latitude=0, longitude=0).astype('datetime64[ns]')
Data_AC_B_mean = np.mean(Data_AC_B['wind_divergence'], axis=(1,2))


licht_mr_data = licht_b_data['mr'].sel(alt=slice(60,5000))
licht_mr_err_data = licht_b_data['mr_err'].sel(alt=slice(60,5000))
coral_mr_data = coral_b_data['mr'].sel(alt=slice(60,5000))
coral_mr_err_data = coral_b_data['mr_err'].sel(alt=slice(60,5000))

### filtering of lidar data:
licht_mr_filtered = FT.noise_reduce(licht_mr_err_data, licht_mr_data, licht_mr_err_data<0.0005)
coral_mr_filtered = FT.noise_reduce(coral_mr_err_data, coral_mr_data, coral_mr_err_data<0.0005)

lidar_combined_data = licht_mr_filtered.combine_first(coral_mr_filtered)


Bco_wind_data = cat.BCO.surfacemet_wxt_v1.to_dask() ### wind data from BCO
wind_speed = Bco_wind_data['VEL']


3. All the necessary datasets are sampled and then saved into memory.

In [ ]:
#AC_B_mean_ws = FT.wind_resample(Data_AC_B_mean, lidar_combined_data, Data_AC_B_time, dist=160000, wind_data=None)
#FT.save_file('AC_B_total_meanWs.nc',AC_B_mean_ws)

AC_B_total = FT.wind_resample(Data_AC_B_mean, lidar_combined_data, Data_AC_B_time, dist=160000, wind_data = wind_speed)
#FT.save_file('AC_B_total.nc',AC_B_total)


### dividing dataset into eastern half and western half

lon_bins = [-60.1875, -59.49, -58.8125]
lon_labels = ['low', 'high']
AC_B_lon = Data_AC_B['longitude']

lon_bingroup = Data_AC_B.groupby_bins(Data_AC_B['longitude'], lon_bins, labels=lon_labels)
west_group = lon_bingroup['low'].mean(dim=('latitude','longitude'), skipna=True)
print(west_group)

east_group = lon_bingroup['high'].mean(dim=('latitude','longitude'), skipna=True)
print(east_group)
#west = FT.wind_resample(west_group, lidar_combined_data, Data_AC_B_time, dist=80000, wind_data = wind_speed)
#east = FT.wind_resample(east_group, lidar_combined_data, Data_AC_B_time, dist=80000, wind_data = wind_speed)


#FT.save_file('AC_B_west.nc', west)
#FT.save_file('AC_B_east.nc', east)

4. Creating of seasonal datasets and saving all into memory

In [ ]:
total_DJF, total_MAM, total_JJA, total_SON = FT.seasons('AC_B_total.nc')
east_DJF, east_MAM, east_JJA, east_SON = FT.seasons('AC_B_east.nc')

FT.save_file('AC_B_total_DJF.nc', total_DJF)
FT.save_file('AC_B_total_MAM.nc', total_MAM)
FT.save_file('AC_B_total_JJA.nc', total_JJA)
FT.save_file('AC_B_total_SON.nc', total_SON)

FT.save_file('AC_B_east_DJF.nc', east_DJF)
FT.save_file('AC_B_east_MAM.nc', east_MAM)
FT.save_file('AC_B_east_JJA.nc', east_JJA)
FT.save_file('AC_B_east_SON.nc', east_SON)

east_AC = xr.open_dataset('AC_B_east.nc')
print(east_AC)

### mean wind speed for each quantile as in table 2:
print(total_DJF.mean(dim='time'))
print(total_MAM.mean(dim='time'))
print(total_JJA.mean(dim='time'))
print(total_SON.mean(dim='time'))

5. Creation of fluctuation plots for figures 1, 2 and 3

In [ ]:
totalQ1, totalQ2, totalQ3, totalQ4 = FT.fluctuation_plots('AC_B_total.nc') ### Figure 1



print('winter, total')
FT.fluctuation_plots('AC_B_total_DJF.nc', season='winter') ### Figure 2a
print('spring, total')
FT.fluctuation_plots('AC_B_total_MAM.nc', season='spring') ### Figure 2b
print('summer, total')
FT.fluctuation_plots('AC_B_total_JJA.nc', season='summer') ### Figure 2c
print('autumn, total')
FT.fluctuation_plots('AC_B_total_SON.nc', season='autumn') ### Figure 2d



print('winter, east')
FT.fluctuation_plots('AC_B_east_DJF.nc', season='winter') ### Figure 3a
print('spring, east')
FT.fluctuation_plots('AC_B_east_MAM.nc', season='spring') ### Figure 3b
print('summer, east')
FT.fluctuation_plots('AC_B_east_JJA.nc', season='summer') ### Figure 3c
print('autumn, east')
FT.fluctuation_plots('AC_B_east_SON.nc', season='autumn') ### Figure 3d



6. Creation of figures 4, 5a and 5b

In [ ]:
FT.compare('AC_B_total.nc', 'AC_B_total_meanWs.nc') ### comparing wind speed methods
FT.compare('AC_B_east.nc', 'AC_B_west.nc') ### comparing eastern half with western half
